In [1]:
import pandas as pd
import datetime as dt

In [ ]:
#link=http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online Retail.xlsx

In [2]:
data=pd.read_excel("Online Retail.xlsx")

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
data["Country"].nunique()

38

In [5]:
data["Country"].value_counts(normalize=True).plot(kind='bar')

In [6]:
data_new=data[data["Country"]=="United Kingdom"].reset_index()

In [7]:
data_new["CustomerID"].nunique(),data["CustomerID"].nunique()

(3950, 4372)

In [8]:
data_new.shape

(495478, 9)

In [9]:
data_new.isnull().sum()

index               0
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [10]:
data_new=data_new[data_new["CustomerID"].isnull()==False]

In [11]:
data_new.isnull().sum()

index          0
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [12]:
data_new.shape

(361878, 9)

In [13]:
sum(data_new["Quantity"]<0)

7533

In [14]:
data_new=data_new[data_new["Quantity"]>=0]

In [15]:
data_new.shape

(354345, 9)

In [16]:
sum(data_new["UnitPrice"]<0)

0

In [17]:
data_new.isnull().sum()

index          0
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [ ]:
# Here we are creating a new column "Total Amount" for monetory score calculation

In [18]:
data_new["Total Amount"]=data_new["UnitPrice"]*data_new["Quantity"]

In [19]:
data_new.CustomerID.nunique()

3921

In [20]:
data_new["InvoiceDate"].max()

Timestamp('2011-12-09 12:49:00')

In [ ]:
# As we can see the latest date is 2011-12-09 so we will take one day after for our calculation

In [21]:
NOW=dt.datetime(2011,12,10)

In [22]:
data_new.dtypes

index                    int64
InvoiceNo               object
StockCode               object
Description             object
Quantity                 int64
InvoiceDate     datetime64[ns]
UnitPrice              float64
CustomerID             float64
Country                 object
Total Amount           float64
dtype: object

In [23]:
RFM_table=data_new.groupby(data_new["CustomerID"]).agg({'InvoiceDate':lambda x: (NOW-x.max()).days,
                                                  'InvoiceNo':lambda x: len(x),
                                                'Total Amount': lambda x: sum(x)})

In [24]:
RFM_table.head()

,InvoiceDate,InvoiceNo,Total Amount
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


In [25]:
# Here we are changing the column names
RFM_table.rename(columns={'InvoiceDate':'Recency',
                   'InvoiceNo':'Frequency',
                   'Total Amount':'Monetory'},inplace=True)

In [26]:
RFM_table.head()

,Recency,Frequency,Monetory
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


In [27]:
RFM_table.describe()

,Recency,Frequency,Monetory
count,3921.000000,3921.000000,3921.000000
mean,91.722265,90.371079,1863.910113
std,99.528532,217.796155,7481.922217
min,0.000000,1.000000,0.000000
25%,17.000000,17.000000,300.040000
50%,50.000000,41.000000,651.820000
75%,142.000000,99.000000,1575.890000
max,373.000000,7847.000000,259657.300000


In [28]:
data_new[data_new["CustomerID"]==12346]

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total Amount
57697,61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6


In [29]:
RFM_table[RFM_table["Frequency"]==7847]

,Recency,Frequency,Monetory
CustomerID,,,
17841.0,1,7847,40991.57


In [30]:
data_new[data_new["CustomerID"]==17841].head(5)

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total Amount
1878,2037,536557,21495,SKULLS AND CROSSBONES WRAP,25,2010-12-01 14:41:00,0.42,17841.0,United Kingdom,10.50
1879,2038,536557,46000R,POLYESTER FILLER PAD 45x30cm,2,2010-12-01 14:41:00,1.45,17841.0,United Kingdom,2.90
1880,2039,536557,46000S,POLYESTER FILLER PAD 40x40cm,1,2010-12-01 14:41:00,1.45,17841.0,United Kingdom,1.45
1881,2040,536557,22731,3D CHRISTMAS STAMPS STICKERS,1,2010-12-01 14:41:00,1.25,17841.0,United Kingdom,1.25
1882,2041,536557,21258,VICTORIAN SEWING BOX LARGE,1,2010-12-01 14:41:00,12.75,17841.0,United Kingdom,12.75


In [31]:
RFM_table["Monetory"]=round(RFM_table["Monetory"],2)

In [32]:
quantiles = RFM_table.quantile(q=[0.25,0.5,0.75])

In [33]:
quantiles=quantiles.to_dict()

In [34]:
quantiles

{'Recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0},
 'Frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0},
 'Monetory': {0.25: 300.04, 0.5: 651.82, 0.75: 1575.89}}

In [35]:
segmented_rfm=RFM_table

In [36]:
def F_score(x,p,d):
    if x<=d[p][0.25]:
        return 1
    elif ((x>d[p][0.25]) & (x<=d[p][0.5])):
        return 2
    elif ((x>d[p][0.5]) & (x<=d[p][0.75])):
        return 3
    else:
        return 4
    
def R_score(x,p,d):
    if x>d[p][0.75]:
        return 1
    elif ((x<=d[p][0.75]) & (x>d[p][0.5])):
        return 2
    elif ((x<=d[p][0.5]) & (x>d[p][0.25])):
        return 3
    else:
        return 4

In [ ]:
#Higher value is giving higher importance to the customer

In [37]:
segmented_rfm["R_score"]=segmented_rfm['Recency'].apply(lambda x : R_score(x,'Recency',quantiles))
segmented_rfm["F_score"]=segmented_rfm['Frequency'].apply(lambda x : F_score(x,'Frequency',quantiles))
segmented_rfm["M_score"]=segmented_rfm['Monetory'].apply(lambda x : F_score(x,'Monetory',quantiles))

In [38]:
segmented_rfm.head()

,Recency,Frequency,Monetory,R_score,F_score,M_score
CustomerID,,,,,,
12346.0,325,1,77183.60,1,1,4
12747.0,2,103,4196.01,4,4,4
12748.0,0,4596,33719.73,4,4,4
12749.0,3,199,4090.88,4,4,4
12820.0,3,59,942.34,4,3,3


In [39]:
segmented_rfm["RFM_score"]=(segmented_rfm["R_score"]*100)+(segmented_rfm["F_score"]*10)+segmented_rfm["M_score"]

In [40]:
segmented_rfm.head()

,Recency,Frequency,Monetory,R_score,F_score,M_score,RFM_score
CustomerID,,,,,,,
12346.0,325,1,77183.60,1,1,4,114
12747.0,2,103,4196.01,4,4,4,444
12748.0,0,4596,33719.73,4,4,4,444
12749.0,3,199,4090.88,4,4,4,444
12820.0,3,59,942.34,4,3,3,433


In [ ]:
# RFM_score 444 is the best customer because it has lower recency,higher frequency,higher monetory

In [41]:
len(segmented_rfm[segmented_rfm["RFM_score"]==444])

409

In [42]:
segmented_rfm[segmented_rfm["RFM_score"]==444].sort_values('Monetory',ascending=False).head()

,Recency,Frequency,Monetory,R_score,F_score,M_score,RFM_score
CustomerID,,,,,,,
18102.0,0,431,259657.30,4,4,4,444
17450.0,8,337,194550.79,4,4,4,444
17511.0,2,963,91062.38,4,4,4,444
16684.0,4,277,66653.56,4,4,4,444
14096.0,4,5111,65164.79,4,4,4,444
